In [1]:
from openmm.app import *
from openmm import *
from openmm.unit import *
from sys import stdout
import MDAnalysis as mda
import nglview as nv
import numpy as np
import pandas as pd
import datetime


In [2]:
# Input pdb
pdb = PDBFile('../data/raw/input.pdb')
forcefield = ForceField('amber14-all.xml', 'amber14/tip3pfb.xml')
system = forcefield.createSystem(pdb.topology, nonbondedMethod=PME, nonbondedCutoff=1*nanometer, constraints=HBonds)
integrator = LangevinMiddleIntegrator(300*kelvin, 1/picosecond, 0.004*picoseconds)
simulation = Simulation(pdb.topology, system, integrator)

# The initial atom positions
simulation.context.setPositions(pdb.positions)
# A local energy minimization
simulation.minimizeEnergy()

# Append the reporters
simulation.reporters.append(PDBReporter('../data/result/output.pbd', 1))
simulation.reporters.append(DCDReporter('../data/result/trajectory.dcd', 1))
simulation.reporters.append(StateDataReporter(stdout, 10, step=True, potentialEnergy=True, temperature=True))

timestep = 10
positions = []
forces = []
# Get postions and forces at each frame
for i in range(timestep):
    simulation.step(1)
    # Create state object
    state = simulation.context.getState(getPositions=True, getForces=True)
    positions.append(state.getPositions(asNumpy=True).value_in_unit(angstrom))
    forces.append(state.getForces(asNumpy=True).value_in_unit(kilojoules/mole/nanometer))

# Save file
dt_now = datetime.datetime.now()
now = dt_now.strftime('%Y%m%d_%H%M%S')
np.save(f"../data/result/positions_{now}", positions)
np.save(f"../data/result/forces_{now}", forces)

#"Step","Potential Energy (kJ/mole)","Temperature (K)"
10,-165931.84218246874,14.700145233862997


In [3]:
np.shape(positions)
print(f"The shape of postions {np.shape(positions)}")
print(f"The shape of forces {np.shape(forces)}")

The shape of postions (10, 8867, 3)
The shape of forces (10, 8867, 3)


In [12]:
positions = np.load("../data/result/forces_20231120_143810.npy")
forces = np.load("../data/result/positions_20231120_143810.npy")

In [3]:
# Visulalize the simulation
md_u = mda.Universe("../examples/input.pdb", "../data/result/trajectory.dcd")
view = nv.show_mdanalysis(md_u)
view

/home/bizon/anaconda3/envs/openmm-junya/lib/python3.9/site-packages/MDAnalysis/topology/PDBParser.py:317: UserWarning: Element information is missing, elements attribute will not be populated. If needed these can be guessed using MDAnalysis.topology.guessers.
  warnings.warn("Element information is missing, elements attribute "


NGLWidget(max_frame=99)